# Model Training and Evaluation
## Import package

In [1]:
from spacy.tokens import DocBin
import spacy
import json
import random

## Convert the data from json format to spaCy data format
### Load json data

In [2]:
with open("../data/ad_data_labeled.json", "r", encoding = "utf-8") as f:
    data = json.load(f)

In [3]:
data[0]

['We could talk about RSL Queensland’s flexible work arrangements, its fully stocked kitchens and fridges and our Benefits Hub.',
 {'entities': [[20, 23, 'SKILL'], [121, 124, 'SKILL']]}]

### Convert data
First of all, I split the processed data into three part(training, testing and validation) with the ratio 4:1:1 and then convert them to .spacy file, which is a new data format in spaCy v3.

In [4]:
def convert_data(data, path):
    nlp = spacy.blank('en')
    db = DocBin()
    for text, annotations in data:
        doc = nlp(text)
        ents = []
        for start, end, label in annotations['entities']:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(path)

training_size = round(len(data)*2/3)
test_size = round((len(data) - training_size)/2)
random.shuffle(data)
TRAIN_DATA = data[0: training_size]
TEST_DATA = data[training_size:training_size+test_size]
VALID_DATA = data[training_size+test_size:]

convert_data(TRAIN_DATA, "../data/train.spacy")
convert_data(TEST_DATA, "../data/test.spacy")
convert_data(VALID_DATA, "../data/dev.spacy")


## Train the model
Instead of python script, spaCy v3 use command line to train the model. The more information can be found [here](https://spacy.io/usage/training#quickstart).

In [ ]:
!python -m spacy train config.cfg --output ../model/NER_spacy_v3

ℹ Saving to output directory: ../model/NER_spacy_v3
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-09 01:26:13,109] [INFO] Set up nlp object from config
[2021-12-09 01:26:13,115] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-12-09 01:26:13,118] [INFO] Created vocabulary
[2021-12-09 01:26:13,119] [INFO] Finished initializing nlp object
[2021-12-09 01:26:16,408] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     36.00    1.91    3.08    1.38    0.02
  0     200        101.76   3523.92   70.24   76.70   64.78    0.70
  0     400        108.73   2239.23   77.68   84.67   71.75    0.78
  0     600        131.11   

## Evaluate the model
Most of skill entities can be found in this model, such as R, Python, coding data and so on. Some mislabeling might happen because the error from scraping data stage, such as learningTimeseries. The errors might also caused from the annotations while data preparation. For example. "IT (information technology)" and "it" may consider as the same word.

In [ ]:
from spacy import displacy
nlp = spacy.load("../model/NER_spacy_v3/model-best")
for i in range(10):
    test_text = TEST_DATA[i][0]
    doc = nlp(test_text)
    displacy.render(doc, style="ent")

The model has 91% accuracy, 89% recall and 90% F1-score, which means it catches the entity well. To improve the performance, the dataset annotation should be done manually instead of rule-based matching.

In [ ]:
!python -m spacy evaluate ../model/NER_spacy_v3/model-best ../data/test.spacy